In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re
import time
import json

options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')

In [2]:
df = pd.DataFrame(pd.read_csv('../output/roll_20_items')).drop(columns=['Unnamed: 0']).drop_duplicates()
df.head()

,item,type,pack_contents,weight,rarity,modifiers,ac,save,stealth,damage,damage_type,duration,secondary_damage,range,properties
0,abacus,adventuring gear,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,acid,adventuring gear (consumable),NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,adamantine armor,"armor (medium or heavy, but not hide)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,alchemist's fire,adventuring gear (consumable),NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,alchemist's supplies,adventuring gear (artisan's tools),NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
item_list = df.item.unique().tolist()
len(item_list)

771

In [4]:
url_prefix = 'https://roll20.net/compendium/dnd5e/'

item_urls = []
for x in item_list:
    item_urls.append(url_prefix + x.replace(' +','%20%2B').replace(' ', '%20'))
    
item_urls

['https://roll20.net/compendium/dnd5e/abacus',
 'https://roll20.net/compendium/dnd5e/acid',
 'https://roll20.net/compendium/dnd5e/adamantine%20armor',
 "https://roll20.net/compendium/dnd5e/alchemist's%20fire",
 "https://roll20.net/compendium/dnd5e/alchemist's%20supplies",
 'https://roll20.net/compendium/dnd5e/amulet',
 'https://roll20.net/compendium/dnd5e/amulet%20of%20health',
 'https://roll20.net/compendium/dnd5e/amulet%20of%20proof%20against%20detection%20and%20location',
 'https://roll20.net/compendium/dnd5e/amulet%20of%20the%20planes',
 'https://roll20.net/compendium/dnd5e/animated%20shield',
 'https://roll20.net/compendium/dnd5e/antitoxin',
 'https://roll20.net/compendium/dnd5e/apparatus%20of%20the%20crab',
 'https://roll20.net/compendium/dnd5e/armor%20of%20invulnerability',
 'https://roll20.net/compendium/dnd5e/armor%20of%20vulnerability',
 'https://roll20.net/compendium/dnd5e/arrow%20of%20slaying',
 'https://roll20.net/compendium/dnd5e/arrow-catching%20shield',
 'https://roll20

In [5]:
def main(item_urls):
    '''add item data to list'''
    item_data = []
    print('SCRAPING ITEM DATA....')
    with tqdm(total=len(item_urls)) as pbar:    
        for index, url in enumerate(item_urls):
            driver = webdriver.Chrome(options=options)
            driver.get(url)
            # get item info
            item_info = driver.find_element(By.ID,value='pagecontent').text
            item_data.append(item_info)
            driver.quit()
            pbar.update(1)
    print('Done.\n\n')
    return item_data

all_items = main(item_urls)
len(all_items)

SCRAPING ITEM DATA....



Done.




771

In [7]:
df['notes'] = all_items
df.to_csv('../output/roll_20_items', index=False)
df

,item,type,pack_contents,weight,rarity,modifiers,ac,save,stealth,damage,damage_type,duration,secondary_damage,range,properties,notes
0,abacus,adventuring gear,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,acid,adventuring gear (consumable),NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"As an action, you can splash the contents of t..."
2,adamantine armor,"armor (medium or heavy, but not hide)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This suit of armor is reinforced with adamanti...
3,alchemist's fire,adventuring gear (consumable),NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"This sticky, adhesive fluid ignites when expos..."
4,alchemist's supplies,adventuring gear (artisan's tools),NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,These Special tools include the items needed t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767,woodcarver's tools,adventuring gear (artisan's tools),NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,These Special tools include the items needed t...
768,wooden shield,shield (shield),NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
769,wooden staff,adventuring gear (arcane focus),NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Druidic Focus: A Druidic focus might be a Spri...
770,yew wand,adventuring gear (druidic focus),NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Druidic Focus: A Druidic focus might be a Spri...
